# CS 416 - Intro to Deep Learning
## Assignment 1

---

**Name** : Syed Muhammad Ashhar Shah

**Reg No** : 2020478

# Task 2

## Importing the required libraries.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

2023-10-13 17:08:13.647210: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 17:08:13.760069: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-13 17:08:13.760117: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-13 17:08:13.760164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-13 17:08:13.771046: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 17:08:15.629324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

## Getting the dataset from the URL

In [2]:
dataset = "./train.csv"

In [3]:
df = pd.read_csv(dataset)

In [4]:
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Selecting Specific Columns
Only using those columns that contain numerical values to make predictions using the linear regression algorithm.

In [5]:
data = df[["MSSubClass", "LotFrontage", "LotArea", "YrSold", "SalePrice"]]

In [6]:
data.head(5)

,MSSubClass,LotFrontage,LotArea,YrSold,SalePrice
0,60,65.0,8450,2008,208500
1,20,80.0,9600,2007,181500
2,60,68.0,11250,2008,223500
3,70,60.0,9550,2006,140000
4,60,84.0,14260,2008,250000


In [7]:
data = data.dropna()

In [8]:
data.dropna()

,MSSubClass,LotFrontage,LotArea,YrSold,SalePrice
0,60,65.0,8450,2008,208500
1,20,80.0,9600,2007,181500
2,60,68.0,11250,2008,223500
3,70,60.0,9550,2006,140000
4,60,84.0,14260,2008,250000
...,...,...,...,...,...
1254,60,60.0,6931,2008,165400
1255,50,52.0,6240,2007,127500
1256,20,91.0,14303,2008,301500
1257,30,56.0,4060,2009,99900


## Converting the dataset to numpy array

In [9]:
dataset_array = np.array(data, dtype=float)
print("Dataset Array Shape: " + str(dataset_array.shape))


X = []
y = []

for row in dataset_array:
  X.append(row[:4])
  y.append(row[4])


X = np.array(X)
y = np.array(y)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

print("X Shape: " + str(X.shape))
print("y Shape: " + str(y.shape))

Dataset Array Shape: (1041, 5)
X Shape: (1041, 4)
y Shape: (1041,)


## Function For Forward Propogation

In [11]:
def forward_prop(w, x):
  y = []
  for row in x:
    y.append(w[0] + row[0]*w[1] + row[1]*w[2] + row[2]*w[3] + row[3]*w[4])

  y = np.array(y)
  return y

## Function For Backward Propogation

In [12]:
def back_prop(w, y, y_hat, X, alpha):
  # get the total number of rows
  n = y.shape[0]
  # create an error against each weight value
  dldw = np.zeros(*w.shape)

  for predict, actual, x in zip(y,y_hat,X):
    error = actual - predict
    dldw[0] = dldw[0] + ((error))
    dldw[1] = dldw[1] + ((error) * x[0])
    dldw[2] = dldw[2] + ((error) * x[1])
    dldw[3] = dldw[3] + ((error) * x[2])
    dldw[4] = dldw[4] + ((error) * x[3])

  w[0] = w[0] + (dldw[0] * alpha)/n
  w[1] = w[1] + (dldw[1] * alpha)/n
  w[2] = w[2] + (dldw[2] * alpha)/n
  w[3] = w[3] + (dldw[3] * alpha)/n
  w[4] = w[4] + (dldw[4] * alpha)/n

  return w

## Cost Function

In [13]:
def cost(y, y_hat):
  n = y.shape[0]

  sum = 0

  for predict, actual in zip(y, y_hat):
    sum = sum + (predict - actual)**2

  j = sum / (2 * n)

  return j

## Carrying out the learning ( ***Schotatic Gradient Descent*** )

### Initialize random weights

In [10]:
weights = np.random.rand(X.shape[1] + 1)

print("Shape of weights: " + str(weights.shape))
print("Random Weights: " + str(weights))

Shape of weights: (5,)
Random Weights: [0.00532489 0.07344137 0.93835135 0.15989516 0.16698177]


### Perfoming Forward and Backward Propgations

In [14]:
costs = []

costs.append(cost(weights, y))

for epoch in range(1000):
  hx = forward_prop(weights, X)
  c = cost(hx, y)
  if epoch % 100 == 0:
    print("Cost: " +str(c))

  costs.append(c)
  weights = back_prop(weights, hx, y, X, 0.01)

Cost: 20130010346.28616
Cost: 5248405681.460156
Cost: 3282386879.3244257
Cost: 3015586089.527865
Cost: 2978353738.1010056
Cost: 2972869321.4951158
Cost: 2971969604.402207
Cost: 2971793535.7238903
Cost: 2971751079.603221
Cost: 2971738944.8056674


### Final Weights For Our Params

In [18]:
weights

array([181645.07808604,   6679.93365795,  28268.76031922,  16899.11687115,
        -3056.93212118])

## Carrying out the learning ( ***Batch Gradient Descent*** )

### Initialize Random Weights

In [20]:
weights = np.random.rand(X.shape[1] + 1)

print("Shape of weights: " + str(weights.shape))
print("Random Weights: " + str(weights))

Shape of weights: (5,)
Random Weights: [0.6414848  0.16813685 0.48364334 0.14921883 0.31575202]
